# Phylogenetic Relatedness Tests

Test phylogenetic signal on microbial community using phylogeny created from rbcL and matK in phylogenerator 20160209_phylogeny.tre.  

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(lsmeans)
library(Hmisc)
library(lme4)
library(lmerTest)
library(ggplot2)
library(dplyr)
library(tidyr)
library(pbkrtest)
library(gridExtra)
library(reshape2)
library(picante)
library(ape)
library(phyloseq)
library(doParallel)


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: estimability

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: lattice

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: survival

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: Formula

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: ggplot2

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/function

# Set up distance matrices

In [3]:
%%R
workDir = '/home/bryan/RhizCG/PhyDist/'
treeFile = '/home/bryan/RhizCG/PhyDist/20160209_phylogeny.tre'
NAMFile = '/home/bryan/RhizCG/PhyDist/NAM_founders_distanceMatrix_v2.txt'

## Phylogenetic distances with expanded within species comparisons

In [4]:
%%R
setwd(workDir)
phy_expand = read.csv(file = "phyDist_expanded.csv", header = TRUE)
colnames(phy_expand)[12] = "75-062"
head(phy_expand)

        FAGES   AMAPO   HELAN   ABUTH   GLYMA   SORBI     B73   ECHCG   SETFA
FAGES 0.00000 0.23210 0.25302 0.27096 0.35879 0.43476 0.43787 0.43985 0.43592
AMAPO 0.23210 0.00000 0.20952 0.22746 0.31529 0.39126 0.39437 0.39635 0.39242
HELAN 0.25302 0.20952 0.00000 0.19116 0.27899 0.35496 0.35807 0.36005 0.35612
ABUTH 0.27096 0.22746 0.19116 0.00000 0.24183 0.35370 0.35681 0.35879 0.35486
GLYMA 0.35879 0.31529 0.27899 0.24183 0.00000 0.44153 0.44464 0.44662 0.44269
SORBI 0.43476 0.39126 0.35496 0.35370 0.44153 0.00000 0.01153 0.03193 0.02800
        ERATE   ELCOR  75-062    Oh7B    MS71     B97     H99   Il14H     Ki3
FAGES 0.44622 0.44745 0.43787 0.43787 0.43787 0.43787 0.43787 0.43787 0.43787
AMAPO 0.40272 0.40395 0.39437 0.39437 0.39437 0.39437 0.39437 0.39437 0.39437
HELAN 0.36642 0.36765 0.35807 0.35807 0.35807 0.35807 0.35807 0.35807 0.35807
ABUTH 0.36516 0.36639 0.35681 0.35681 0.35681 0.35681 0.35681 0.35681 0.35681
GLYMA 0.45299 0.45422 0.44464 0.44464 0.44464 0.44464 0.44464 0.

In [5]:
%%R 
setwd(workDir)
NAMdist = read.table(NAMFile, header = TRUE)

#reorder so that NAMdist is in same order as log2fold change matrix
NAMdist = NAMdist[c(1:6, 8, 7, 9, 10), c(1:6, 8, 7, 9, 10)]

### Use melt to create a dataframe with species by species comparisons that will later be compared with plot by plot unifrac distances

In [6]:
%%R
pd.m = melt(as.matrix(phy_expand))

#set column names and create comparison column
colnames(pd.m) = c("PlantA", "PlantB", "phy_Dist")
pd.m$PlantComp = paste(pd.m$PlantA, pd.m$PlantB, sep = "-")
pd.m %>% head


  PlantA PlantB phy_Dist   PlantComp
1  FAGES  FAGES  0.00000 FAGES-FAGES
2  AMAPO  FAGES  0.23210 AMAPO-FAGES
3  HELAN  FAGES  0.25302 HELAN-FAGES
4  ABUTH  FAGES  0.27096 ABUTH-FAGES
5  GLYMA  FAGES  0.35879 GLYMA-FAGES
6  SORBI  FAGES  0.43476 SORBI-FAGES


## unifrac distances

In [7]:
%%R
F = file.path('/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/physeq/Full-Sparsity3in3_r')
physeq.Full = readRDS(F)



physeq.Full = filter_taxa(physeq.Full, function(x) sum(x > 0) > 0, TRUE)
physeq.Nneg = subset_samples(physeq.Full, Plant != "BARE" & Treatment == "Nneg")
physeq.Nneg   

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 4982 taxa and 87 samples ]
sample_data() Sample Data:       [ 87 samples by 55 sample variables ]
tax_table()   Taxonomy Table:    [ 4982 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 4982 tips and 4981 internal nodes ]


In [8]:
%%R
registerDoParallel(cores=28)
wunif.dist = distance(physeq.Nneg, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

In [9]:
%%R
as.matrix(wunif.dist) %>% head

                  sample5-1.22_Nneg sample4-2.02_Nneg sample4-4.21_Nneg
sample5-1.22_Nneg         0.0000000         0.2069072         0.1605861
sample4-2.02_Nneg         0.2069072         0.0000000         0.1853471
sample4-4.21_Nneg         0.1605861         0.1853471         0.0000000
sample5-2.29_Nneg         0.1505018         0.1881794         0.1816279
sample5-2.06_Nneg         0.1332870         0.2056788         0.1890167
sample1-2.23_Nneg         0.1635179         0.2520001         0.1917811
                  sample5-2.29_Nneg sample5-2.06_Nneg sample1-2.23_Nneg
sample5-1.22_Nneg         0.1505018         0.1332870         0.1635179
sample4-2.02_Nneg         0.1881794         0.2056788         0.2520001
sample4-4.21_Nneg         0.1816279         0.1890167         0.1917811
sample5-2.29_Nneg         0.0000000         0.1162380         0.1563019
sample5-2.06_Nneg         0.1162380         0.0000000         0.1837943
sample1-2.23_Nneg         0.1563019         0.1837943         0.

## Add identifiers and join tables 

In [10]:
%%R
sd = sample_data(physeq.Nneg)
colnames(sd)

 [1] "X.sample"            "Library"             "Primer_Plate"       
 [4] "Primer_Well_ID"      "primerFR_ID_byPlate" "primerFR_ID_total"  
 [7] "barcode_ID"          "PlotID"              "Plant"              
[10] "Plot"                "Treatment"           "Rep"                
[13] "DAP"                 "Sampling"            "PlantSample"        
[16] "Type"                "Grass_Broad"         "Status"             
[19] "Subfamily"           "fwd_barcode"         "rev_barcode"        
[22] "Planted"             "Biomass_P"           "SLA"                
[25] "Leaf_N"              "Leaf_C"              "LeafC_N"            
[28] "LNa"                 "Plant_N"             "Plant_C"            
[31] "Plant_CN"            "d15N"                "tN_p"               
[34] "BX_activity_g"       "CB_activity_g"       "LAP_activity_g"     
[37] "NAG_activity_g"      "BX_rel"              "CB_rel"             
[40] "LAP_rel"             "NAG_rel"             "NO3_ug_g"           
[43] "

In [11]:
%%R


ud.m = melt(as.matrix(wunif.dist))
head(ud.m) %>% print
dim(ud.m) %>% print
colnames(ud.m) = c("PlotA", "PlotB", "w_Unif_Dist")

#Join with plant names
ud.m = left_join(ud.m, sd[, c(1,9)], by = c("PlotA" = "X.sample"))
head(ud.m) %>% print
colnames(ud.m)[4] = "PlantA"

ud.m = left_join(ud.m, sd[, c(1,9)], by = c("PlotB" = "X.sample"))
head(ud.m) %>% print
colnames(ud.m)[5] = "PlantB"
ud.m$PlantComp = paste(ud.m$PlantA, ud.m$PlantB, sep = "-")

#Join wit days after planting
ud.m = left_join(ud.m, sd[, c(1,13)], by = c("PlotA" = "X.sample"))

colnames(ud.m)[7] = "DateA"

ud.m = left_join(ud.m, sd[, c(1,13)], by = c("PlotB" = "X.sample"))
head(ud.m) %>% print
colnames(ud.m)[8] = "DateB"

#Time distance variable
ud.m$TimeDist = abs(ud.m$DateB - ud.m$DateA)

head(ud.m)


               Var1              Var2     value
1 sample5-1.22_Nneg sample5-1.22_Nneg 0.0000000
2 sample4-2.02_Nneg sample5-1.22_Nneg 0.2069072
3 sample4-4.21_Nneg sample5-1.22_Nneg 0.1605861
4 sample5-2.29_Nneg sample5-1.22_Nneg 0.1505018
5 sample5-2.06_Nneg sample5-1.22_Nneg 0.1332870
6 sample1-2.23_Nneg sample5-1.22_Nneg 0.1635179
[1] 7569    3
              PlotA             PlotB w_Unif_Dist  Plant
1 sample5-1.22_Nneg sample5-1.22_Nneg   0.0000000 75-062
2 sample4-2.02_Nneg sample5-1.22_Nneg   0.2069072  SETFA
3 sample4-4.21_Nneg sample5-1.22_Nneg   0.1605861  SETFA
4 sample5-2.29_Nneg sample5-1.22_Nneg   0.1505018 75-062
5 sample5-2.06_Nneg sample5-1.22_Nneg   0.1332870   MS71
6 sample1-2.23_Nneg sample5-1.22_Nneg   0.1635179  FAGES
              PlotA             PlotB w_Unif_Dist PlantA  Plant
1 sample5-1.22_Nneg sample5-1.22_Nneg   0.0000000 75-062 75-062
2 sample4-2.02_Nneg sample5-1.22_Nneg   0.2069072  SETFA 75-062
3 sample4-4.21_Nneg sample5-1.22_Nneg   0.1605861  SETFA 75

In [12]:
%%R
#Create plot-plot variable for joining with functional distance matrix
ud.m$PlotComp = paste(ud.m$PlotA, ud.m$PlotB, sep = "-")
head(ud.m) %>% print

              PlotA             PlotB w_Unif_Dist PlantA PlantB     PlantComp
1 sample5-1.22_Nneg sample5-1.22_Nneg   0.0000000 75-062 75-062 75-062-75-062
2 sample4-2.02_Nneg sample5-1.22_Nneg   0.2069072  SETFA 75-062  SETFA-75-062
3 sample4-4.21_Nneg sample5-1.22_Nneg   0.1605861  SETFA 75-062  SETFA-75-062
4 sample5-2.29_Nneg sample5-1.22_Nneg   0.1505018 75-062 75-062 75-062-75-062
5 sample5-2.06_Nneg sample5-1.22_Nneg   0.1332870   MS71 75-062   MS71-75-062
6 sample1-2.23_Nneg sample5-1.22_Nneg   0.1635179  FAGES 75-062  FAGES-75-062
  DateA DateB TimeDist                            PlotComp
1    72    72        0 sample5-1.22_Nneg-sample5-1.22_Nneg
2    61    72       11 sample4-2.02_Nneg-sample5-1.22_Nneg
3    61    72       11 sample4-4.21_Nneg-sample5-1.22_Nneg
4    72    72        0 sample5-2.29_Nneg-sample5-1.22_Nneg
5    72    72        0 sample5-2.06_Nneg-sample5-1.22_Nneg
6    36    72       36 sample1-2.23_Nneg-sample5-1.22_Nneg


In [13]:
%%R
head(ud.m)

              PlotA             PlotB w_Unif_Dist PlantA PlantB     PlantComp
1 sample5-1.22_Nneg sample5-1.22_Nneg   0.0000000 75-062 75-062 75-062-75-062
2 sample4-2.02_Nneg sample5-1.22_Nneg   0.2069072  SETFA 75-062  SETFA-75-062
3 sample4-4.21_Nneg sample5-1.22_Nneg   0.1605861  SETFA 75-062  SETFA-75-062
4 sample5-2.29_Nneg sample5-1.22_Nneg   0.1505018 75-062 75-062 75-062-75-062
5 sample5-2.06_Nneg sample5-1.22_Nneg   0.1332870   MS71 75-062   MS71-75-062
6 sample1-2.23_Nneg sample5-1.22_Nneg   0.1635179  FAGES 75-062  FAGES-75-062
  DateA DateB TimeDist                            PlotComp
1    72    72        0 sample5-1.22_Nneg-sample5-1.22_Nneg
2    61    72       11 sample4-2.02_Nneg-sample5-1.22_Nneg
3    61    72       11 sample4-4.21_Nneg-sample5-1.22_Nneg
4    72    72        0 sample5-2.29_Nneg-sample5-1.22_Nneg
5    72    72        0 sample5-2.06_Nneg-sample5-1.22_Nneg
6    36    72       36 sample1-2.23_Nneg-sample5-1.22_Nneg


## Join unifrac and plant phylogenetic distance; PrimeDist can be used with other subsets of data

In [14]:
%%R
PrimeDist = left_join(ud.m, pd.m[,c(3,4)], by = "PlantComp")


# add self/non-self comparisons
PrimeDist$Self =  ifelse(PrimeDist$phy_Dist == 0, 0, 1)
        

tail(PrimeDist) %>% print
    

                 PlotA             PlotB w_Unif_Dist PlantA PlantB   PlantComp
7564 sample8-1.23_Nneg sample8-3.03_Nneg   0.1984166    Ki3  ELCOR   Ki3-ELCOR
7565 sample6-2.14_Nneg sample8-3.03_Nneg   0.2503097  Hp301  ELCOR Hp301-ELCOR
7566 sample8-4.04_Nneg sample8-3.03_Nneg   0.2377478   Mo18  ELCOR  Mo18-ELCOR
7567 sample3-3.30_Nneg sample8-3.03_Nneg   0.2233484  ABUTH  ELCOR ABUTH-ELCOR
7568 sample7-4.14_Nneg sample8-3.03_Nneg   0.2893173  SORBI  ELCOR SORBI-ELCOR
7569 sample8-3.03_Nneg sample8-3.03_Nneg   0.0000000  ELCOR  ELCOR ELCOR-ELCOR
     DateA DateB TimeDist                            PlotComp phy_Dist Self
7564    88    88        0 sample8-1.23_Nneg-sample8-3.03_Nneg  0.05090    1
7565    79    88        9 sample6-2.14_Nneg-sample8-3.03_Nneg  0.05090    1
7566    88    88        0 sample8-4.04_Nneg-sample8-3.03_Nneg  0.05090    1
7567    57    88       31 sample3-3.30_Nneg-sample8-3.03_Nneg  0.36639    1
7568    84    88        4 sample7-4.14_Nneg-sample8-3.03_Nneg  0.04

# Mantel test of phylogenetic distance and microbial community distance

## Set up distance matrices

In [15]:
%%R
colnames(PrimeDist)

 [1] "PlotA"       "PlotB"       "w_Unif_Dist" "PlantA"      "PlantB"     
 [6] "PlantComp"   "DateA"       "DateB"       "TimeDist"    "PlotComp"   
[11] "phy_Dist"    "Self"       


In [16]:
%%R
head(PrimeDist) %>% print
#Remove NAs to trim dataset
#Temp = Dist[!is.na(Dist$phy_Dist),]
#dim(Temp) %>% print

#Seperate unifrac and phylogenetic distances retaining plot IDs
U = PrimeDist[,c(1,2,3)]
Phy = PrimeDist[,c(1,2,11)]
Self = PrimeDist[,c(1,2,12)]
T = PrimeDist[,c(1,2,9)]

#Re-cast longform to wide form.  

Udist = acast(U, PlotA~PlotB)
Phydist = acast(Phy, PlotA~PlotB)
Sdist = acast(Self, PlotA~PlotB)
Tdist = acast(T, PlotA~PlotB) 

#Convert to distance matrices
Phydist = as.matrix(Phydist) 
Udist = as.matrix(Udist)  
Sdist = as.matrix(Sdist) 
Tdist = as.matrix(Tdist) 

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Using w_Unif_Dist as value column: use value.var to override.

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Using phy_Dist as value column: use value.var to override.

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Using Self as value column: use value.var to override.

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Using TimeDist as value column: use value.var to override.

  res = super(Function, self).__call__(*new_args, **new_kwargs)


              PlotA             PlotB w_Unif_Dist PlantA PlantB     PlantComp
1 sample5-1.22_Nneg sample5-1.22_Nneg   0.0000000 75-062 75-062 75-062-75-062
2 sample4-2.02_Nneg sample5-1.22_Nneg   0.2069072  SETFA 75-062  SETFA-75-062
3 sample4-4.21_Nneg sample5-1.22_Nneg   0.1605861  SETFA 75-062  SETFA-75-062
4 sample5-2.29_Nneg sample5-1.22_Nneg   0.1505018 75-062 75-062 75-062-75-062
5 sample5-2.06_Nneg sample5-1.22_Nneg   0.1332870   MS71 75-062   MS71-75-062
6 sample1-2.23_Nneg sample5-1.22_Nneg   0.1635179  FAGES 75-062  FAGES-75-062
  DateA DateB TimeDist                            PlotComp phy_Dist Self
1    72    72        0 sample5-1.22_Nneg-sample5-1.22_Nneg  0.00000    0
2    61    72       11 sample4-2.02_Nneg-sample5-1.22_Nneg  0.03111    1
3    61    72       11 sample4-4.21_Nneg-sample5-1.22_Nneg  0.03111    1
4    72    72        0 sample5-2.29_Nneg-sample5-1.22_Nneg  0.00000    0
5    72    72        0 sample5-2.06_Nneg-sample5-1.22_Nneg  0.00020    1
6    36    72   

In [17]:
%%R
head(Sdist)

                  sample1-1.27_Nneg sample1-2.23_Nneg sample1-3.08_Nneg
sample1-1.27_Nneg                 0                 0                 0
sample1-2.23_Nneg                 0                 0                 0
sample1-3.08_Nneg                 0                 0                 0
sample1-4.24_Nneg                 0                 0                 0
sample2-1.08_Nneg                 1                 1                 1
sample2-1.24_Nneg                 1                 1                 1
                  sample1-4.24_Nneg sample2-1.08_Nneg sample2-1.24_Nneg
sample1-1.27_Nneg                 0                 1                 1
sample1-2.23_Nneg                 0                 1                 1
sample1-3.08_Nneg                 0                 1                 1
sample1-4.24_Nneg                 0                 1                 1
sample2-1.08_Nneg                 1                 0                 1
sample2-1.24_Nneg                 1                 1           

In [18]:
%%R
#check congruence
Udist %>% as.matrix %>% head


                  sample1-1.27_Nneg sample1-2.23_Nneg sample1-3.08_Nneg
sample1-1.27_Nneg         0.0000000        0.14661893        0.13776227
sample1-2.23_Nneg         0.1466189        0.00000000        0.09806129
sample1-3.08_Nneg         0.1377623        0.09806129        0.00000000
sample1-4.24_Nneg         0.1314250        0.12007763        0.11602099
sample2-1.08_Nneg         0.3519226        0.30109695        0.31696808
sample2-1.24_Nneg         0.2608532        0.24885430        0.24903303
                  sample1-4.24_Nneg sample2-1.08_Nneg sample2-1.24_Nneg
sample1-1.27_Nneg         0.1314250         0.3519226         0.2608532
sample1-2.23_Nneg         0.1200776         0.3010969         0.2488543
sample1-3.08_Nneg         0.1160210         0.3169681         0.2490330
sample1-4.24_Nneg         0.0000000         0.3421983         0.2727548
sample2-1.08_Nneg         0.3421983         0.0000000         0.3209266
sample2-1.24_Nneg         0.2727548         0.3209266         0.

# Mantel test, including all plants and maize with arbitrary 0.0002 distance between maize



In [22]:
%%R

M1 = ecodist::mantel(as.dist(Udist) ~ as.dist(Tdist) , nperm=999)
M1

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.291430032 0.001001001 1.000000000 0.001001001 0.263474870 0.323048115 


In [21]:
%%R

M_TP = ecodist::mantel(as.dist(Phydist) ~ as.dist(TimeDist) , nperm=999)
M_TP


Error in as.dist(TimeDist) : object 'TimeDist' not found


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in as.dist(TimeDist) : object 'TimeDist' not found

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: In addition: 
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Warning messages:

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 1: replacing previous import ‘BiocGenerics::Position’ by ‘ggplot2::Position’ when loading ‘phyloseq’ 

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 2: 
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/

## Partial Mantel test of phylogenetic distance controlling for self/non-self

In [59]:
%%R
M2 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist) + as.dist(Sdist) , nperm=999)
M2

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.351130873 0.001001001 1.000000000 0.001001001 0.293158532 0.397382590 


In [60]:
%%R
M3 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist) + as.dist(Tdist) + as.dist(Sdist) , nperm=999)
M3

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.259588546 0.001001001 1.000000000 0.001001001 0.207923844 0.311145927 


In [54]:
%%R
M4 = ecodist::mantel(as.dist(Udist) ~ as.dist(Tdist), nperm=999)
M4

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.291430032 0.001001001 1.000000000 0.001001001 0.263530344 0.322840552 


## Mantel test with species comparisons to test for maize effect on phylogenetic results

In [24]:
%%R
ADist =  filter(PrimeDist, PlantA %in% c( "ABUTH" , "AMAPO", "ECHCG" , "ERATE", "FAGES",  "GLYMA", "SETFA", "SORBI",   
                                    "SORSUD", "ELCOR","HELAN", "B73") & PlantB %in% 
                                    
                c("ABUTH" , "AMAPO", "ECHCG" , "ERATE", "FAGES",  "GLYMA", "SETFA", "SORBI",   
                                    "SORSUD", "ELCOR","HELAN", "B73"))


#Seperate unifrac and phylogenetic distances retaining plot IDs
U = ADist[,c(1,2,3)]
Phy = ADist[,c(1,2,11)]
Self = ADist[,c(1,2,12)]
T = ADist[,c(1,2,9)]

#Re-cast longform to wide form.  

Udist = acast(U, PlotA~PlotB)
Phydist = acast(Phy, PlotA~PlotB)
Sdist = acast(Self, PlotA~PlotB)
Tdist = acast(T, PlotA~PlotB) 

#Convert to distance matrices
Phydist = as.matrix(Phydist) 
Udist = as.matrix(Udist)  
Sdist = as.matrix(Sdist) 
Tdist = as.matrix(Tdist) 

In [25]:
%%R
M1 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist), nperm=999)
M1

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.225519198 0.002002002 0.998998999 0.002002002 0.179131261 0.268626623 


In [26]:
%%R
M2 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist)  + as.dist(Sdist), nperm=999)
M2

   mantelr      pval1      pval2      pval3  llim.2.5% ulim.97.5% 
0.11857321 0.03803804 0.96296296 0.05005005 0.05593426 0.17077105 


In [27]:
%%R
M3 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist) + as.dist(Tdist) + as.dist(Sdist), nperm=999)
M3


    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
 0.04016677  0.23423423  0.76676677  0.49149149 -0.03342176  0.09600982 


In [28]:
%%R
M4 = ecodist::mantel(as.dist(Phydist) ~ as.dist(Tdist), nperm=999)
M4

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.375183045 0.001001001 1.000000000 0.001001001 0.309145165 0.454877292 


In [29]:
%%R
M5 = ecodist::mantel(as.dist(Phydist) ~ as.dist(Tdist) + as.dist(Phydist) + as.dist(Sdist), nperm=999)
M5

    mantelr       pval1       pval2       pval3   llim.2.5%  ulim.97.5% 
0.283012350 0.001001001 1.000000000 0.001001001 0.206723204 0.360003105 


* Weighted unifrac distance is postiviely correlated with phylogenetic distance between hosts
* Robust when accounting for within genotype comparisons.  
* Robust when using only one maize line
* Not robust when controlling for time in reduced dataset as time and phylogenetic distance are positively correlated 

### Notes: 
    * If DAP is included as a functional trait functional variation explains an equal portion of the variance as phylogenetic distance:  Mantel r = 0.326 p = 0.001
    * If DAP is not included, considerably less variation, though still significant Mantel R = 0.136 and p = 0.001
    * I should try including both N treatments to broaden variation

## Early Season Annuals Mantel Test

In [ ]:
%%R
# Cut distances to comparisons among early season plants
EDist =  filter(PrimeDist, PlantA %in% c("FAGES", "ECHCG", "AMAPO", "SETFA", "GLYMA",
                                                 "ABUTH") & PlantB %in% 
                                    
                c("FAGES", "ECHCG", "AMAPO", "SETFA", "GLYMA",
                                                 "ABUTH"))


#Seperate unifrac and phylogenetic distances retaining plot IDs
U = EDist[,c(1,2,3)]
Phy = EDist[,c(1,2,11)]
Self = EDist[,c(1,2,12)]
T = EDist[,c(1,2,9)]

#Re-cast longform to wide form.  

Udist = acast(U, PlotA~PlotB)
Phydist = acast(Phy, PlotA~PlotB)
Sdist = acast(Self, PlotA~PlotB)
Tdist = acast(T, PlotA~PlotB) 

#Convert to distance matrices
Phydist = as.matrix(Phydist) 
Udist = as.matrix(Udist)  
Sdist = as.matrix(Sdist) 
Tdist = as.matrix(Tdist) 

In [ ]:
%%R
M1 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist), nperm=999)
M1

In [ ]:
%%R
M2 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist)  + as.dist(Sdist), nperm=999)
M2

In [ ]:
%%R
M3 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist) + as.dist(Tdist) + as.dist(Sdist), nperm=999)
M3


## Late season annuals Mantel test

In [ ]:
%%R
# Cut distances to comparisons among early season plants
EDist =  filter(PrimeDist, PlantA %in% c("B73", "HELAN", "ERATE", "SORSUD",
                                                 "SORBI", "ELCOR") & PlantB %in% 
                                    
                c("B73", "HELAN", "ERATE", "SORSUD",
                                                 "SORBI", "ELCOR"))


#Seperate unifrac and phylogenetic distances retaining plot IDs
U = EDist[,c(1,2,3)]
Phy = EDist[,c(1,2,11)]
Self = EDist[,c(1,2,12)]
T = EDist[,c(1,2,9)]

#Re-cast longform to wide form.  

Udist = acast(U, PlotA~PlotB)
Phydist = acast(Phy, PlotA~PlotB)
Sdist = acast(Self, PlotA~PlotB)
Tdist = acast(T, PlotA~PlotB) 

#Convert to distance matrices
Phydist = as.matrix(Phydist) 
Udist = as.matrix(Udist)  
Sdist = as.matrix(Sdist) 
Tdist = as.matrix(Tdist) 

In [ ]:
%%R
M1 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist), nperm=999)
M1

In [ ]:
%%R
M2 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist)  + as.dist(Sdist), nperm=999)
M2

In [ ]:
%%R
M3 = ecodist::mantel(as.dist(Udist) ~ as.dist(Phydist) + as.dist(Tdist) + as.dist(Sdist), nperm=999)
M3

# Maize distance analysis

In [ ]:
%%R
NAM_hc = hclust(as.dist(NAMdist), "average")
plot(NAM_hc)

In [ ]:
%%R
physeq.Maize = subset_samples(physeq.Full, Type == "Maize" & Plant != "75-062" & Treatment == "Nneg")
as(sample_data(physeq.Maize), "data.frame") %>%

group_by(Plant) %>%
   summarise(DATE = first(DAP)) %>% 
    arrange(DATE) %>%
   print()

In [ ]:
%%R
#Use melt to join genetic distance matrix with unifrac distance
#head(ud.m) ##Use formated unifrac distance from above
#melt 
NAM.m = melt(as.matrix(NAMdist))

#set column names and create comparison column
colnames(NAM.m) = c("PlantA", "PlantB", "gene_Dist")

NAM.m$PlantComp = paste(NAM.m$PlantA, NAM.m$PlantB, sep = "-")
NAM.m %>% head

#Join with unifrac DistanceMatrix
GeneDist = left_join(ud.m, NAM.m[,c(3,4)], by = "PlantComp") %>% filter(!is.na(gene_Dist))
head(GeneDist)

#add self/non-self comparisons
GeneDist$Self =  ifelse(GeneDist$gene_Dist == 0, 0, 1)
        

tail(GeneDist) %>% print

In [ ]:
%%R

#Seperate unifrac and phylogenetic distances retaining plot IDs
U = GeneDist[,c(1,2,3)]
Gene = GeneDist[,c(1,2,11)]
Self = GeneDist[,c(1,2,12)]
T = GeneDist[,c(1,2,9)]

#Re-cast longform to wide form.  

Udist = acast(U, PlotA~PlotB)
Genedist = acast(Gene, PlotA~PlotB)
Sdist = acast(Self, PlotA~PlotB)
Tdist = acast(T, PlotA~PlotB) 

#Convert to distance matrices
Gdist = as.matrix(Phydist) 
Udist = as.matrix(Udist)  
Sdist = as.matrix(Sdist) 
Tdist = as.matrix(Tdist) 

## Overall maize genetic distance vs MCC

In [ ]:
%%R
M1 = ecodist::mantel(as.dist(Udist)~as.dist(Gdist), nperm=999)
print(M1)

In [ ]:
%%R
M2 = ecodist::mantel(as.dist(Udist) ~ as.dist(Gdist) + as.dist(Sdist), nperm=999)
M2

In [ ]:
%%R
M3 = ecodist::mantel(as.dist(Udist) ~ as.dist(Gdist) + as.dist(Tdist) + as.dist(Sdist), nperm=999)
M3


In [ ]:
%%R
ggplot(GeneDist, aes(gene_Dist, w_Unif_Dist)) + 
geom_point()

### Functional comparison of maize - Mantel test

In [ ]:
%%R
sd = sample_data(physeq.Maize)

# Calculate w-unifrac distances
registerDoParallel(cores=28)
maize.wunif.dist = distance(physeq.Maize, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

#Calculate functional distances
Fun_dist = dist(sd[,c("Resid_logB", "Resid_logN", "Resid_logCN")], method = "euclidian")


M1 = mantel(maize.wunif.dist, Fun_dist, method="pearson", permutations=999)
M1

In test of genetic vs functional control of maize, growth residuals were not significantly correlated with microbial community similarity in mantel test.  Significance was gained by including DAP, however, this is now capturing day to day sampling variation.  

### Functional comparison of maize, CAP

In [ ]:
%%R -w 600 -h 600
phy = physeq.Maize

sample_data(phy)$DAP = as.factor(sample_data(phy)$DAP)
unique(sample_data(phy)$DAP) %>% print
# CAP ordinate
cap.ord = ordinate(
    physeq = phy, 
    method = "CAP",
    distance = maize.wunif.dist,
    formula = ~ Resid_logB + Resid_logN + Resid_logCN + Condition(Library)
)

# CAP plot
cap.plot <- plot_ordination(
    phy, 
    cap.ord, 
    color = "Plant", 
    axes = c(1,2)
    ) + 
    aes(shape = Type) + 
    geom_point(aes(colour = Plant), 
        alpha= 1, 
        size = 4) + 
    #geom_point(colour = "grey90", 
    #    size = 1.5) + 
    theme_bw(base_size = 16)
    #scale_color_manual(values = c("#a65628", "red", "#ffae19", "#4daf4a", 
    #    "#1919ff", "darkorchid3", "magenta"))

# Now add the environmental variables as arrows
arrowmat <- vegan::scores(cap.ord, display = "bp")


# Add labels, make a data.frame
arrowdf <- data.frame(labels = rownames(arrowmat), arrowmat)

# Define the arrow aesthetic mapping
arrow_map <- aes(xend = CAP1, 
    yend = CAP2, 
    x = 0, 
    y = 0, 
    shape = NULL, 
    color = NULL, 
    label = labels)

label_map <- aes(x = 1.3 * CAP1, 
    y = 1.3 * CAP2, 
    shape = NULL, 
    color = NULL, 
    label = labels)

arrowhead = arrow(length = unit(0.02, "npc"))

# Make a new graphic
cap.plot + 
geom_segment(arrow_map, 
    size = .5, 
    data = arrowdf, 
    color = "gray", 
    arrow = arrowhead
    ) + 
geom_text(label_map, 
    size = 4,  
    data = arrowdf, 
    show_guide = FALSE)

In [ ]:
%%R
anova(cap.ord)

Notes - a small but significant amount of variation in community composition is captured by maize functional variation.  

Check how to constrain by DAP to control for differences from sampling date?  Still significant albeit small portion of variation captured when controlling for DAP.  

### Maize by date

#### DAP = 72; Mantel test maize genetic distance vs MCC

In [ ]:
%%R

# subset matrices
D_dist = filter(Dist, PlantA %in% c("H99" , "Il14H" , "MS71") & PlantB %in% c("H99" , "Il14H" , "MS71"))
D_dist$PlantComp %>% unique %>% print
#pull matrices
    Temp = D_dist[!is.na(D_dist$gene_Dist),]
    
    #Seperate unifrac and genetic distances retaining plot IDs

    Udate = Temp[,c(1,2,3)]
    Gene_date = Temp[,c(1,2,7)]

    #Re-cast longform to wide form.  
    Udate = acast(Udate, PlotA~PlotB)
    Gene_date = acast(Gene_date, PlotA~PlotB)

    #Convert to distance matrices
    Gene_date = as.matrix(Gene_date) %>% dist()
    Udate = as.matrix(Udate) %>% dist()

M1 = mantel(Gene_date, Udate, method="pearson", permutations=999)
print(M1)

#### DAP = 72; Mantel test N economy traits vs MCC

In [ ]:
%%R
physeq.Date = subset_samples(physeq.Plant, Plant %in% c("H99" , "Il14H" , "MS71"))
sd = sample_data(physeq.Date)

# Calculate w-unifrac distances
registerDoParallel(cores=28)
date.wunif.dist = distance(physeq.Date, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

#Calculate functional distances
Fun_dist = dist(sd[,c("Resid_logB", "Resid_logN", "Resid_logCN")], method = "euclidian")


M1 = mantel(date.wunif.dist, Fun_dist, method="pearson", permutations=999)
M1

#### DAP 72; CAP analysis N economy and MCC

In [ ]:
%%R -w 600 -h 600
phy = physeq.Date

sample_data(phy)$DAP = as.factor(sample_data(phy)$DAP)
unique(sample_data(phy)$DAP) %>% print
# CAP ordinate
cap.ord = ordinate(
    physeq = phy, 
    method = "CAP",
    distance = date.wunif.dist,
    formula = ~ Resid_logB + Resid_logN + Resid_logCN
)

# CAP plot
cap.plot <- plot_ordination(
    phy, 
    cap.ord, 
    color = "Plant", 
    axes = c(1,2)
    ) + 
    aes(shape = Type) + 
    geom_point(aes(colour = Plant), 
        alpha= 1, 
        size = 4) + 
    #geom_point(colour = "grey90", 
    #    size = 1.5) + 
    theme_bw(base_size = 16)
    #scale_color_manual(values = c("#a65628", "red", "#ffae19", "#4daf4a", 
    #    "#1919ff", "darkorchid3", "magenta"))

# Now add the environmental variables as arrows
arrowmat <- vegan::scores(cap.ord, display = "bp")


# Add labels, make a data.frame
arrowdf <- data.frame(labels = rownames(arrowmat), arrowmat)

# Define the arrow aesthetic mapping
arrow_map <- aes(xend = CAP1, 
    yend = CAP2, 
    x = 0, 
    y = 0, 
    shape = NULL, 
    color = NULL, 
    label = labels)

label_map <- aes(x = 1.3 * CAP1, 
    y = 1.3 * CAP2, 
    shape = NULL, 
    color = NULL, 
    label = labels)

arrowhead = arrow(length = unit(0.02, "npc"))

# Make a new graphic
cap.plot + 
geom_segment(arrow_map, 
    size = .5, 
    data = arrowdf, 
    color = "gray", 
    arrow = arrowhead
    ) + 
geom_text(label_map, 
    size = 4,  
    data = arrowdf, 
    show_guide = FALSE)

In [ ]:
%%R
anova(cap.ord)

#### DAP = 79;  Mantel test maize genetic distance vs MCC

In [ ]:
%%R

# subset matrices
D_dist = filter(Dist, PlantA %in% c("B73" ,  "B97" ,  "Hp301" , "Oh7B" ) & PlantB %in% c("B73" ,  "B97" ,  "Hp301" , "Oh7B"))
 D_dist$PlantComp %>% unique %>% print

#pull matrices
    Temp = D_dist[!is.na(D_dist$gene_Dist),]
    
    #Seperate unifrac and genetic distances retaining plot IDs

    Udate = Temp[,c(1,2,3)]
    Gene_date = Temp[,c(1,2,7)]

    #Re-cast longform to wide form.  
    Udate = acast(Udate, PlotA~PlotB)
    Gene_date = acast(Gene_date, PlotA~PlotB)

    #Convert to distance matrices
    Gene_date = as.matrix(Gene_date) %>% dist()
    Udate = as.matrix(Udate) %>% dist()

M1 = mantel(Gene_date, Udate, method="pearson", permutations=999)
print(M1)

#### DAP = 79;  Mantel test N economy traits vs MCC

In [ ]:
%%R
physeq.Date = subset_samples(physeq.Plant, Plant %in% c("B73" ,  "B97" ,  "Hp301" , "Oh7B"))
sd = sample_data(physeq.Date)
D_dist$PlantComp %>% unique %>% print
# Calculate w-unifrac distances
registerDoParallel(cores=28)
date.wunif.dist = distance(physeq.Date, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

#Calculate functional distances
Fun_dist = dist(sd[,c("Resid_logB", "Resid_logN", "Resid_logCN")], method = "euclidian")


M1 = mantel(date.wunif.dist, Fun_dist, method="pearson", permutations=999)
M1

#### DAP 79; CAP analysis N economy vs MCC

In [ ]:
%%R -w 600 -h 600
phy = physeq.Date

sample_data(phy)$DAP = as.factor(sample_data(phy)$DAP)
unique(sample_data(phy)$DAP) %>% print
# CAP ordinate
cap.ord = ordinate(
    physeq = phy, 
    method = "CAP",
    distance = date.wunif.dist,
    formula = ~ Resid_logB + Resid_logN + Resid_logCN + Condition(Library)
)

# CAP plot
cap.plot <- plot_ordination(
    phy, 
    cap.ord, 
    color = "Plant", 
    axes = c(1,2)
    ) + 
    aes(shape = Type) + 
    geom_point(aes(colour = Plant), 
        alpha= 1, 
        size = 4) + 
    #geom_point(colour = "grey90", 
    #    size = 1.5) + 
    theme_bw(base_size = 16)
    #scale_color_manual(values = c("#a65628", "red", "#ffae19", "#4daf4a", 
    #    "#1919ff", "darkorchid3", "magenta"))

# Now add the environmental variables as arrows
arrowmat <- vegan::scores(cap.ord, display = "bp")


# Add labels, make a data.frame
arrowdf <- data.frame(labels = rownames(arrowmat), arrowmat)

# Define the arrow aesthetic mapping
arrow_map <- aes(xend = CAP1, 
    yend = CAP2, 
    x = 0, 
    y = 0, 
    shape = NULL, 
    color = NULL, 
    label = labels)

label_map <- aes(x = 1.3 * CAP1, 
    y = 1.3 * CAP2, 
    shape = NULL, 
    color = NULL, 
    label = labels)

arrowhead = arrow(length = unit(0.02, "npc"))

# Make a new graphic
cap.plot + 
geom_segment(arrow_map, 
    size = .5, 
    data = arrowdf, 
    color = "gray", 
    arrow = arrowhead
    ) + 
geom_text(label_map, 
    size = 4,  
    data = arrowdf, 
    show_guide = FALSE)

In [ ]:
%%R
anova(cap.ord)

#### DAP = 88; Mantel test maize genetic distance vs MCC

In [ ]:
%%R

# subset matrices
D_dist = filter(Dist, PlantA %in% c("Ki3" ,  "Mo18" , "Tx303" ) & PlantB %in% c("Ki3" ,  "Mo18" , "Tx303"))
D_dist$PlantComp %>% unique %>% print
#pull matrices
    Temp = D_dist[!is.na(D_dist$gene_Dist),]
    
    #Seperate unifrac and genetic distances retaining plot IDs

    Udate = Temp[,c(1,2,3)]
    Gene_date = Temp[,c(1,2,7)]

    #Re-cast longform to wide form.  
    Udate = acast(Udate, PlotA~PlotB)
    Gene_date = acast(Gene_date, PlotA~PlotB)

    #Convert to distance matrices
    Gene_date = as.matrix(Gene_date) %>% dist()
    Udate = as.matrix(Udate) %>% dist()

M1 = mantel(Gene_date, Udate, method="pearson", permutations=999)
print(M1)

#### DAP = 88; Mantel test N economy vs MCC

In [ ]:
%%R
physeq.Date = subset_samples(physeq.Plant, Plant %in% c("Ki3" ,  "Mo18" , "Tx303"))
sd = sample_data(physeq.Date)

# Calculate w-unifrac distances
registerDoParallel(cores=28)
date.wunif.dist = distance(physeq.Date, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

#Calculate functional distances
Fun_dist = dist(sd[,c("Resid_logB", "Resid_logN", "Resid_logCN")], method = "euclidian")


M1 = mantel(date.wunif.dist, Fun_dist, method="pearson", permutations=999)
M1

#### DAP = 88; CAP analysis N economy vs MCC

In [ ]:
%%R -w 600 -h 600
phy = physeq.Date

sample_data(phy)$DAP = as.factor(sample_data(phy)$DAP)
unique(sample_data(phy)$DAP) %>% print
# CAP ordinate
cap.ord = ordinate(
    physeq = phy, 
    method = "CAP",
    distance = date.wunif.dist,
    formula = ~ Resid_logB + Resid_logN + Resid_logCN
)

# CAP plot
cap.plot <- plot_ordination(
    phy, 
    cap.ord, 
    color = "Plant", 
    axes = c(1,2)
    ) + 
    aes(shape = Type) + 
    geom_point(aes(colour = Plant), 
        alpha= 1, 
        size = 4) + 
    #geom_point(colour = "grey90", 
    #    size = 1.5) + 
    theme_bw(base_size = 16)
    #scale_color_manual(values = c("#a65628", "red", "#ffae19", "#4daf4a", 
    #    "#1919ff", "darkorchid3", "magenta"))

# Now add the environmental variables as arrows
arrowmat <- vegan::scores(cap.ord, display = "bp")


# Add labels, make a data.frame
arrowdf <- data.frame(labels = rownames(arrowmat), arrowmat)

# Define the arrow aesthetic mapping
arrow_map <- aes(xend = CAP1, 
    yend = CAP2, 
    x = 0, 
    y = 0, 
    shape = NULL, 
    color = NULL, 
    label = labels)

label_map <- aes(x = 1.3 * CAP1, 
    y = 1.3 * CAP2, 
    shape = NULL, 
    color = NULL, 
    label = labels)

arrowhead = arrow(length = unit(0.02, "npc"))

# Make a new graphic
cap.plot + 
geom_segment(arrow_map, 
    size = .5, 
    data = arrowdf, 
    color = "gray", 
    arrow = arrowhead
    ) + 
geom_text(label_map, 
    size = 4,  
    data = arrowdf, 
    show_guide = FALSE)

In [ ]:
%%R
anova(cap.ord)

# Time vs MCC mantel test

In [31]:
%%R
sample_data(physeq.Nneg)$DAP

 [1] 72 61 61 72 72 36 79 79 72 53 72 57 72 36 72 79 61 79 72 79 72 79 72 72 57
[26] 53 72 79 72 79 72 72 57 79 79 79 72 72 53 79 72 36 61 72 72 36 57 53 88 84
[51] 88 84 57 79 84 84 53 88 79 53 88 88 84 84 88 57 84 88 84 84 84 84 53 88 88
[76] 88 88 88 88 79 57 88 79 88 57 84 88


In [ ]:
%%R
sd = sample_data(physeq.Nneg)

# Calculate w-unifrac distances
registerDoParallel(cores=28)
date.wunif.dist = distance(physeq.Nneg, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

#Calculate functional distances
Fun_dist = dist(sd[,c("DAP")], method = "euclidian")


M1 = mantel(date.wunif.dist, Fun_dist, method="pearson", permutations=999)
M1

In [ ]:
%%R
mcc.correlog <- mantel.correlog(date.wunif.dist, Fun_dist, nperm=99)
summary(mcc.correlog) %>% print
mcc.correlog  %>% print
plot(mcc.correlog) 

# Ignore below this point

# Compare plant phylogenetic distance to l2f

In [ ]:
%%R
# log2foldFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/DeSeq2/bare-plant.csv'
# l2f = read.csv(log2foldFile, header = TRUE)
# colnames(l2f)

### Spread by plant to make matrix of log2fold change in each OTU by plant combo

In [ ]:
%%R
# df = l2f[,c(1,11,13)]
# l2f_m = spread(df, Plant, log2FoldChange, fill = 0, drop = TRUE)
# rownames(l2f_m) = l2f_m$OTU
# l2f_m = l2f_m[,-1]
# l2f_m %>% head

In [ ]:
%%R
# dim(l2f_m)

### recode all negative log2fold changes with 0

In [ ]:
%%R
# l2f_me = l2f_m
# l2f_me[l2f_me < 0] = 0 
# head(l2f_me)

## Calculate pairwise weighted Unifrac distances between samples

need to figure out how to run unifrac measure outside of phyloseq

In [ ]:
%%R
# physeq.Full

In [ ]:
%%R
# #provide fast tree for unifrac calculation
# #treeFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/fasttree/otusn.tree'
# #Tree = read.tree(treeFile)
# Tree = phy_tree(physeq.Full)
# str(Tree)


In [ ]:
%%R
# l2f_wunif = picante::unifrac(as.matrix(t(l2f_me)), Tree)

In [ ]:
%%R
# l2f_bray = vegdist(as.matrix(t(l2f_me)), method = "bray")

In [ ]:
%%R
# hc_unif = hclust(l2f_wunif, "average")
# plot(hc_unif) %>% print

# hc_bray = hclust(l2f_bray, "average")
# plot(hc_bray, main = "Cluster Dendrogram, \n Bray-Curtis Distance")

# Mantel test of phylogenetic distance and l2f unifrac distance

### Maize

In [ ]:
%%R
# NAMdist %>% print

# #subset column and rows of log2fold change so that only NAM lines are included
# NLU = as.matrix(l2f_wunif)
# NLU = NLU[c(4,5,11,13:18,22),c(4,5,11,13:18,22)]
# NLU

# NLB = as.matrix(l2f_bray)
# NLB = NLB[c(4,5,11,13:18,22),c(4,5,11,13:18,22)]
# NLB

In [ ]:
%%R
# M1 = mantel(as.dist(NAMdist), as.dist(NLU), method="pearson", permutations=999)
# print(M1)

In [ ]:
%%R
# M1 = mantel(as.dist(NAMdist), as.dist(NLB), method="pearson", permutations=999)
# print(M1)

### Species

In [ ]:
%%R

# #Organize log2fold change distance matrix so that only plants with cholorplast distances are included 
# #and column and row order is congruent with phylogenetic distance matrix
# l2f_wunif.t = as.matrix(l2f_wunif) %>% as.data.frame()
#     temp1 = l2f_wunif.t[c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR"), c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR")]

#     SLU = as.matrix(temp1)

#     SLU = as.dist(SLU)
#     SLU %>% print

# #Remove ginkgo so that matrices are congruent
# p.temp = as.matrix(phydist) %>% as.data.frame()
#     temp2 = p.temp[-12, -12] #remove ginkgo

#     temp2 = as.matrix(temp2)
#     phydist = as.dist(temp2)

#     phydist %>% print

# l2f_bray.t = as.matrix(l2f_bray) %>% as.data.frame()
#     temp2 = l2f_bray.t[c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR"), c("FAGES", "AMAPO", "HELAN", "ABUTH", "GLYMA", "SORBI", "B73", "ECHCG",
#                     "SETFA", "ERATE", "ELCOR")]

#     SLB = as.matrix(temp2)
#     SLB = as.dist(SLB)
#     SLB %>% print


In [ ]:
%%R
# m1 = mantel(phydist, SLU, method="pearson", permutations=999)
# print(m1)

In [ ]:
%%R
# m1 = mantel(phydist, SLB, method="pearson", permutations=999)
# print(m1)